In [54]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [55]:
data = pd.read_csv('Data/prepared_data.csv').drop(['Unnamed: 0'], axis = 1)

In [56]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [57]:
data['Studio'] = data['Studio'].apply(clean_data)

In [58]:
genres_ = ['Безумие', 'Боевые искусства',
       'Вампиры', 'Военное', 'Гарем', 'Демоны', 'Детектив', 'Детское',
       'Дзёсэй', 'Драма', 'Игры', 'Исторический', 'Комедия', 'Космос', 'Магия',
       'Машины', 'Меха', 'Музыка', 'Пародия', 'Повседневность', 'Полиция',
       'Приключения', 'Психологическое', 'Романтика', 'Самураи',
       'Сверхъестественное', 'Сёдзё', 'Сёдзё Ай', 'Сёнэн', 'Сёнэн-Aй', 'Спорт',
       'Супер сила', 'Сэйнэн', 'Триллер', 'Ужасы', 'Фантастика', 'Фэнтези',
       'Школа', 'Экшен', 'Этти']
age_ = ['Age_6', 'Age_12', 'Age_16', 'Age_18']
type_ = ['Type_ONA', 'Type_OVA', 'Type_Спешл', 'Type_ТВ Сериал', 'Type_Фильм']

In [59]:
def join_ages(x):
    a = ''
    for g in age_:
        if x[g] == 1:
            a += g+' '
    return a

In [60]:
def join_genres(x):
    gen = ''
    for g in genres_:
        if x[g] == 1:
            gen += 'g_'+g+' '
    return gen

In [61]:
def join_types(x):
    types = ''
    for t in type_:
        if x[t] == 1:
            types += t+' '
    return types

In [62]:
data['genres'] = data.apply(join_genres, axis=1)
data['types'] = data.apply(join_types, axis=1)
data['types'] = data['types'].apply(clean_data)
data['ages'] = data.apply(join_ages, axis=1)

In [63]:
data.columns

Index(['Url', 'Poster', 'Title', 'Synonym', 'Description', 'Year', 'Studio',
       'Duration', 'Rating', 'Num votes', 'Безумие', 'Боевые искусства',
       'Вампиры', 'Военное', 'Гарем', 'Демоны', 'Детектив', 'Детское',
       'Дзёсэй', 'Драма', 'Игры', 'Исторический', 'Комедия', 'Космос', 'Магия',
       'Машины', 'Меха', 'Музыка', 'Пародия', 'Повседневность', 'Полиция',
       'Приключения', 'Психологическое', 'Романтика', 'Самураи',
       'Сверхъестественное', 'Сёдзё', 'Сёдзё Ай', 'Сёнэн', 'Сёнэн-Aй', 'Спорт',
       'Супер сила', 'Сэйнэн', 'Триллер', 'Ужасы', 'Фантастика', 'Фэнтези',
       'Школа', 'Экшен', 'Этти', 'Type_ONA', 'Type_OVA', 'Type_Спешл',
       'Type_ТВ Сериал', 'Type_Фильм', 'Age_6', 'Age_12', 'Age_16', 'Age_18',
       'score', 'genres', 'types', 'ages'],
      dtype='object')

In [64]:
data

,Url,Poster,Title,Synonym,Description,Year,Studio,Duration,Rating,Num votes,...,Type_ТВ Сериал,Type_Фильм,Age_6,Age_12,Age_16,Age_18,score,genres,types,ages
0,https://animego.org/anime/klinok-rassekayuschi...,https://animego.org/media/cache/thumbs_250x350...,"Клинок, рассекающий демонов: Квартал красных ф...",Kimetsu no Yaiba: Yuukaku-hen,Продолжение приключений Тандзиро и его неизмен...,2022,ufotable,23,9.6,29503,...,1,0,0,0,0,1,9.532862,g_Демоны g_Исторический g_Сверхъестественное g...,type_твсериал,Age_18
1,https://animego.org/anime/klinok-rassekayuschi...,https://animego.org/media/cache/thumbs_250x350...,"Клинок, рассекающий демонов",Kimetsu no Yaiba,Эпоха «Тайсё» была полна неожиданностей. Много...,2019,ufotable,23,9.5,36813,...,1,0,0,0,0,1,9.447877,g_Демоны g_Исторический g_Сверхъестественное g...,type_твсериал,Age_18
2,https://animego.org/anime/reinkarnaciya-bezrab...,https://animego.org/media/cache/thumbs_250x350...,Реинкарнация безработного: История о приключен...,Mushoku Tensei: Isekai Ittara Honki Dasu,«В этом новом мире я возьмусь за голову!» Встр...,2021,studiobind,23,9.5,28754,...,1,0,0,0,0,1,9.433629,g_Драма g_Магия g_Фэнтези g_Этти,type_твсериал,Age_18
3,https://animego.org/anime/ohotnik-h-ohotnik-2-280,https://animego.org/media/cache/thumbs_250x350...,Охотник х Охотник 2,Hunter x Hunter (2011),"Юноша Гон, обитающий на Китовом острове, мечта...",2014,madhousestudios,23,9.6,10994,...,1,0,0,0,1,0,9.426861,g_Приключения g_Сёнэн g_Фэнтези g_Экшен,type_твсериал,Age_16
4,https://animego.org/anime/reinkarnaciya-bezrab...,https://animego.org/media/cache/thumbs_250x350...,Реинкарнация безработного: История о приключен...,Mushoku Tensei: Isekai Ittara Honki Dasu Part 2,"Эта история расскажет об отаку-неудачнике, кот...",2021,studiobind,23,9.5,24897,...,1,0,0,0,0,1,9.423640,g_Драма g_Фэнтези g_Этти,type_твсериал,Age_18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982,https://animego.org/anime/nevesta-titana-1593,https://animego.org/media/cache/thumbs_250x350...,Невеста титана,Kyojinzoku no Hanayome,Высокий мускулистый Коити выгодно отличался от...,2020,0,6,5.4,1923,...,1,0,0,0,0,1,5.789173,g_Сёнэн-Aй g_Фэнтези,type_твсериал,Age_18
1983,https://animego.org/anime/shkola-mezhdu-izmere...,https://animego.org/media/cache/thumbs_250x350...,Школа между измерениями,Dimension High School,"В школах от ребят требуют трудолюбия, усидчиво...",2019,0,10,2.7,274,...,1,0,0,0,1,0,5.691413,g_Школа,type_твсериал,Age_16
1984,https://animego.org/anime/sd-gandam-geroi-mira...,https://animego.org/media/cache/thumbs_250x350...,СД Гандам: Герои мира,SD Gundam World Heroes,"На Нео Мир рухнул метеорит, вызвав повсюду хао...",2021,sunrise,24,2.3,264,...,0,0,0,1,0,0,5.615193,,type_ona,Age_12
1985,https://animego.org/anime/zametki-tesly-1889,https://animego.org/media/cache/thumbs_250x350...,Заметки Теслы,Tesla Note,Их цель — выполнить Т-миссию и спасти мир! Дуэ...,2021,0,24,2.9,439,...,1,0,0,1,0,0,5.343996,g_Экшен,type_твсериал,Age_12


In [65]:
def create_soup_1(x):
    return x['types'] + str(x['Year']) + ' ' + str(x['Duration']) + ' ' + x['ages']
def create_soup_2(x):
    return x['Studio'] + ' ' + x['genres']

In [66]:
data['soup_1'] = data.apply(create_soup_1, axis=1)

In [67]:
tfidf = TfidfVectorizer()
count_soup = CountVectorizer()
count_genres = CountVectorizer()
count_studio = CountVectorizer()

In [68]:
tfidf_matrix = tfidf.fit_transform(data['Description'])
soup_matrix = count_soup.fit_transform(data['soup_1'])
genres_matrix = count_soup.fit_transform(data['genres'])
studio_matrix = count_soup.fit_transform(data['Studio'])

In [69]:
cosine_discription = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_soup = cosine_similarity(soup_matrix, soup_matrix)
cosine_genres = cosine_similarity(genres_matrix, genres_matrix)
cosine_studio = cosine_similarity(studio_matrix, studio_matrix)

In [70]:
cos_metric = (cosine_discription*0.4 + cosine_soup*0.2 + cosine_genres*0.45 + cosine_studio*0.15)

In [71]:
indices = pd.Series(data.index, index=data['Title']).drop_duplicates()

In [72]:
def get_recommendations(title, cosine_sim=cos_metric):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[0:20]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return data['Title'].iloc[movie_indices]

In [73]:
get_recommendations('Охотник х Охотник 2')

3                                     Охотник х Охотник 2
221                                     Охотник х Охотник
223                                    Такт. Опус Дестини
1525                                    Исчезнувший город
1433                                     Магические войны
806                 Заводной дух войны: Альдерамин в небе
82                                          Убийца Акамэ!
784                                Драконий жемчуг: Супер
398                               Маги: Королевство магии
1980                       Боруто: Новое поколение Наруто
1894    Каким-то образом я стал сильнейшим, прокачивая...
1274                     Сильнейший мудрец низшей эмблемы
533                                       Далёкий паладин
1891                         Маг-обманщик из другого мира
1410                                       Комета Люцифер
1292                                   Магмел синего моря
1931                   Президент, пришло время для битвы!
215           